## Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 11868
%cd /content/drive/MyDrive/11868
!git clone https://github.com/llmsystem/llmsys_code_examples.git
%cd /content/drive/MyDrive/11868/llmsys_code_examples/tensor_demo/miniTorch

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install -r requirements.extra.txt

In [ ]:
!pip install -Ue .

In [5]:
import minitorch
import numpy as np

## Indexing

<img src="../imgs/strides.png"></img>

In [6]:
x = minitorch.tensor([1, 2, 3, 4, 5, 6])

In [7]:
x.to_numpy()

array([1., 2., 3., 4., 5., 6.])

In [9]:
x._tensor.shape, x._tensor.strides

(6,) (1,)


In [45]:
y = minitorch.Tensor.make(
    storage=x._tensor._storage, 
    shape=(2, 3), 
    strides=(3, 1),
    backend=x.backend)
y


[
	[1.000000 2.000000 3.000000]
	[4.000000 5.000000 6.000000]]

In [46]:
y._tensor._storage

array([1., 2., 3., 4., 5., 6.])

In [47]:
y._tensor._storage is x._tensor._storage

True

In [55]:
z = x.view(3, 2, 1)
z


[
	[
		[1.000000]
		[2.000000]]
	[
		[3.000000]
		[4.000000]]
	[
		[5.000000]
		[6.000000]]]

In [56]:
z._tensor.shape, y._tensor.strides

((3, 2, 1), (3, 1))

In [57]:
z._tensor._storage is x._tensor._storage

True

In [61]:
z_index = [1, 1, 0]
pos = minitorch.index_to_position(z_index, z._tensor._strides)
z[tuple(z_index)] == z._tensor._storage[pos], pos, z[tuple(z_index)]

(True, 3, 4.0)

In [62]:
out_index = [0, 0, 0]
minitorch.to_index(3, z.shape, out_index)
out_index

[1, 1, 0]

In [63]:
shape = (2, 3)
minitorch.strides_from_shape(shape)

(3, 1)

In [64]:
shape = (3, 2, 1)
minitorch.strides_from_shape(shape)

(2, 1, 1)

In [66]:
p = z.permute(2, 1, 0)
p


[
	[
		[1.000000 3.000000 5.000000]
		[2.000000 4.000000 6.000000]]]

In [67]:
p._tensor.shape, p._tensor.strides

((1, 2, 3), (1, 1, 2))

In [68]:
p._tensor._storage is x._tensor._storage

True

In [69]:
p_index = [0, 0, 0]
minitorch.to_index(-1, p.shape, p_index)
p_index

[0, 1, 2]

In [70]:
minitorch.index_to_position(p_index, p._tensor._strides)

5

## Shape Broadcast

### Broadcast rules


<img src="../imgs/broadcast_rule.png"></img>

### Broadcast: Vector / Matrix + Scalar


<img src="../imgs/broadcast_mv_scalar.png"></img>

In [93]:
"""
Implemented in minitorch/tensor.py

def _ensure_tensor(self, b: TensorLike) -> Tensor:
    "Turns a python number into a tensor with the same backend."
    if isinstance(b, (int, float)):
        c = Tensor.make([b], (1,), backend=self.backend)
    else:
        b._type_(self.backend)
        c = b
    return c

def __add__(self, b: TensorLike) -> Tensor:
    return Add.apply(self, self._ensure_tensor(b))
"""

a1 = minitorch.tensor([1, 2, 3, 4, 5, 6])
b = a1 + 10
b, b.shape

(
 [11.000000 12.000000 13.000000 14.000000 15.000000 16.000000],
 (6,))

In [82]:
a2 = a1._ensure_tensor(10)
a2, a2._tensor.shape, a2._tensor.strides

(
 [10.000000],
 (1,),
 (1,))

In [87]:
out_shape = minitorch.shape_broadcast(a1.shape, a2.shape)
out_shape

(6,)

In [90]:
out_index = [0] * len(out_shape)
a1_index = [0] * len(out_shape)
a2_index = [0] * len(out_shape)


minitorch.to_index(1, out_shape, out_index)
o = minitorch.index_to_position(out_index, b._tensor.strides)
print(f"out_index: {out_index}, o: {o}")
minitorch.broadcast_index(out_index, out_shape, a1.shape, a1_index)
i1 = minitorch.index_to_position(a1_index, a1._tensor.strides)
print(f"a1_index: {a1_index}, i1: {i1}")
minitorch.broadcast_index(out_index, out_shape, a2.shape, a2_index)
i2 = minitorch.index_to_position(a2_index, a2._tensor.strides)
print(f"a2_index: {a2_index}, i2: {i2}")

out_index: [1], o: 1
a1_index: [1], i1: 1
a2_index: [0], i2: 0


In [92]:
c1 = minitorch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
d = c1 + 10
d, d.shape

(
 [
 	[
 		[11.000000]
 		[12.000000]
 		[13.000000]]
 	[
 		[14.000000]
 		[15.000000]
 		[16.000000]]],
 (2, 3, 1))

In [94]:
c2 = c1._ensure_tensor(10)
c2, c2._tensor.shape, c2._tensor.strides

(
 [10.000000],
 (1,),
 (1,))

In [125]:
out_index = [0] * len(d.shape)
c1_index = [0] * len(d.shape)
c2_index = [0] * len(d.shape)


minitorch.to_index(4, d.shape, out_index)
o = minitorch.index_to_position(out_index, d._tensor.strides)
print(f"out_index: {out_index}, o: {o}, d[out_index]: {d[tuple(out_index)]}")
minitorch.broadcast_index(out_index, d.shape, c1.shape, c1_index)
i1 = minitorch.index_to_position(c1_index, c1._tensor.strides)
print(f"c1_index: {c1_index[:len(c1.shape)]}, i1: {i1}, c1[c1_index]: {c1[tuple(c1_index[:len(c1.shape)])]}")
minitorch.broadcast_index(out_index, out_shape, c2.shape, c2_index)
i2 = minitorch.index_to_position(c2_index, c2._tensor.strides)
print(f"c2_index: {c2_index[:len(c2.shape)]}, i2: {i2}, c2[c2_index]: {c2[tuple(c2_index[:len(c2.shape)])]}")

out_index: [1, 1, 0], o: 4, d[out_index]: 15.0
c1_index: [1, 1, 0], i1: 4, c1[c1_index]: 5.0
c2_index: [0], i2: 0, c2[c2_index]: 10.0


### Broadcast: Matrix + Vector


<img src="../imgs/broadcast_matrix_vector.png"></img>

In [137]:
e1 = minitorch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
e2 = minitorch.tensor([1, 2, 3])
f = e1 + e2
f, f.shape

(
 [
 	[2.000000 4.000000 6.000000]
 	[5.000000 7.000000 9.000000]],
 (2, 3))

In [138]:
f_shape = minitorch.shape_broadcast(e1.shape, e2.shape)
f_shape

(2, 3)

In [139]:
f._tensor._storage, f._tensor._storage[4]

(array([2., 4., 6., 5., 7., 9.]), 7.0)

In [141]:
out_index = [0] * len(f.shape)
e1_index = [0] * len(f.shape)
e2_index = [0] * len(f.shape)


minitorch.to_index(4, f.shape, out_index)
o = minitorch.index_to_position(out_index, f._tensor.strides)
print(f"out_index: {out_index}, o: {o}, f[out_index]: {f[tuple(out_index)]}")
minitorch.broadcast_index(out_index, f.shape, e1.shape, e1_index)
i1 = minitorch.index_to_position(e1_index, e1._tensor.strides)
print(f"e1_index: {e1_index[:len(e1.shape)]}, i1: {i1}, e1[e1_index]: {e1[tuple(e1_index[:len(e1.shape)])]}")
minitorch.broadcast_index(out_index, f.shape, e2.shape, e2_index)
i2 = minitorch.index_to_position(e2_index, e2._tensor.strides)
print(f"e2_index: {e2_index[:len(e2.shape)]}, i2: {i2}, e2[e2_index]: {e2[tuple(e2_index[:len(e2.shape)])]}")

out_index: [1, 1], o: 4, f[out_index]: 7.0
e1_index: [1, 1], i1: 4, e1[e1_index]: 5.0
e2_index: [1], i2: 1, e2[e2_index]: 2.0


In [142]:
f._tensor._storage, f._tensor._storage[1]

(array([2., 4., 6., 5., 7., 9.]), 4.0)

In [144]:
out_index = [0] * len(f.shape)
e1_index = [0] * len(f.shape)
e2_index = [0] * len(f.shape)


minitorch.to_index(0, f.shape, out_index)
o = minitorch.index_to_position(out_index, f._tensor.strides)
print(f"out_index: {out_index}, o: {o}, f[out_index]: {f[tuple(out_index)]}")
minitorch.broadcast_index(out_index, f.shape, e1.shape, e1_index)
i1 = minitorch.index_to_position(e1_index, e1._tensor.strides)
print(f"e1_index: {e1_index[:len(e1.shape)]}, i1: {i1}, e1[e1_index]: {e1[tuple(e1_index[:len(e1.shape)])]}")
minitorch.broadcast_index(out_index, f.shape, e2.shape, e2_index)
i2 = minitorch.index_to_position(e2_index, e2._tensor.strides)
print(f"e2_index: {e2_index[:len(e2.shape)]}, i2: {i2}, e2[e2_index]: {e2[tuple(e2_index[:len(e2.shape)])]}")

out_index: [0, 0], o: 0, f[out_index]: 2.0
e1_index: [0, 0], i1: 0, e1[e1_index]: 1.0
e2_index: [0], i2: 0, e2[e2_index]: 1.0


## Reduce Demo

<img src="../imgs/reduce.jpg"></img>

In [6]:
t = minitorch.tensor([i for i in range(24)]).view(2, 3, 4)

In [4]:
t


[
	[
		[0.000000 1.000000 2.000000 3.000000]
		[4.000000 5.000000 6.000000 7.000000]
		[8.000000 9.000000 10.000000 11.000000]]
	[
		[12.000000 13.000000 14.000000 15.000000]
		[16.000000 17.000000 18.000000 19.000000]
		[20.000000 21.000000 22.000000 23.000000]]]

In [3]:
t.sum(1)


[
	[
		[12.000000 15.000000 18.000000 21.000000]]
	[
		[48.000000 51.000000 54.000000 57.000000]]]

In [15]:
reduce_dim = 1
reduce_value = 0
out_shape = list(t.shape)
out_shape[reduce_dim] = 1
out_storage = [reduce_value] * np.prod(out_shape)
out_strides = minitorch.strides_from_shape(out_shape)
in_storage = t._tensor._storage
in_strides = t._tensor._strides

In [9]:
fn = lambda x, y: x + y

In [20]:
out_index = [0] * 3
reduce_size = t.shape[reduce_dim]
for i in range(len(out_storage)):
    minitorch.to_index(i, out_shape, out_index)
    o = minitorch.index_to_position(out_index, out_strides)
    print(f"out_idx: {i}, out_index: {out_index}, out_pos: {o}")
    for s in range(reduce_size):
        out_index[reduce_dim] = s
        j = minitorch.index_to_position(out_index, in_strides)
        print(f"a_index: {out_index}, a_pos: {j}")
        out_storage[o] = fn(out_storage[o], in_storage[j])

out_idx: 0, out_index: [0, 0, 0], out_pos: 0
a_index: [0, 0, 0], a_pos: 0
a_index: [0, 1, 0], a_pos: 4
a_index: [0, 2, 0], a_pos: 8
out_idx: 1, out_index: [0, 0, 1], out_pos: 1
a_index: [0, 0, 1], a_pos: 1
a_index: [0, 1, 1], a_pos: 5
a_index: [0, 2, 1], a_pos: 9
out_idx: 2, out_index: [0, 0, 2], out_pos: 2
a_index: [0, 0, 2], a_pos: 2
a_index: [0, 1, 2], a_pos: 6
a_index: [0, 2, 2], a_pos: 10
out_idx: 3, out_index: [0, 0, 3], out_pos: 3
a_index: [0, 0, 3], a_pos: 3
a_index: [0, 1, 3], a_pos: 7
a_index: [0, 2, 3], a_pos: 11
out_idx: 4, out_index: [1, 0, 0], out_pos: 4
a_index: [1, 0, 0], a_pos: 12
a_index: [1, 1, 0], a_pos: 16
a_index: [1, 2, 0], a_pos: 20
out_idx: 5, out_index: [1, 0, 1], out_pos: 5
a_index: [1, 0, 1], a_pos: 13
a_index: [1, 1, 1], a_pos: 17
a_index: [1, 2, 1], a_pos: 21
out_idx: 6, out_index: [1, 0, 2], out_pos: 6
a_index: [1, 0, 2], a_pos: 14
a_index: [1, 1, 2], a_pos: 18
a_index: [1, 2, 2], a_pos: 22
out_idx: 7, out_index: [1, 0, 3], out_pos: 7
a_index: [1, 0, 3], 

In [17]:
out_storage

[12.0, 15.0, 18.0, 21.0, 48.0, 51.0, 54.0, 57.0]